In [25]:
import pandas as pd

# Load the CSV file (Replace with your actual file path)
df = pd.read_csv("your_local_path/Output_Areas_2021-UK.csv")

# Select only the required columns
selected_columns = ["OA21CD", "LAT", "LONG"]
new_df = df[selected_columns]

# Rename the columns
new_df = new_df.rename(columns={"OA21CD": "OA 2021", "LAT": "Lat", "LONG": "Long"})

# Save the new dataset
new_df.to_csv("UK-OA.csv", index=False)

print("New dataset created successfully: UK-OA.csv")


New dataset created successfully: UK-OA.csv


In [30]:
import pandas as pd

def merge_datasets(file1, file2, output_file):
    # Load the datasets
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)

    # Strip column names to remove leading/trailing spaces
    df1.columns = df1.columns.str.strip()
    df2.columns = df2.columns.str.strip()

    # Rename column if necessary
    if 'OA 2021' in df1.columns:
        df1.rename(columns={'OA 2021': 'OA2021'}, inplace=True)
    if 'OA 2021' in df2.columns:
        df2.rename(columns={'OA 2021': 'OA2021'}, inplace=True)

    # Check if 'OA2021' exists
    if 'OA2021' not in df1.columns or 'OA2021' not in df2.columns:
        raise KeyError("Column 'OA2021' not found in one or both files. Check column names.")

    # Remove any missing values in 'OA2021'
    df1.dropna(subset=['OA2021'], inplace=True)
    df2.dropna(subset=['OA2021'], inplace=True)

    # Merge datasets on 'OA2021'
    merged_df = pd.merge(df1, df2, on='OA2021', how='inner')

    # Select relevant columns
    merged_df = merged_df[['OA2021', 'Lat', 'Long', 'Population']]

    # Save the merged dataset
    merged_df.to_csv(output_file, index=False)
    print(f"Merged dataset saved to {output_file}")

# Example usage
file1 = "your_local_path/UK-OA.csv"
file2 = "your_local_path/Population-OA-2022.csv"
output_file = "your_local_path/UK-OA-Population.csv"

merge_datasets(file1, file2, output_file)


Merged dataset saved to D:/your_local_path/UK-OA-Population.csv


In [31]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Load litter data (assumes a CSV file with 'latitude' and 'longitude' columns)
litter_df = pd.read_csv("your_local_path/southampton_data.csv")  # Change to your actual file

# Convert litter data into a GeoDataFrame
litter_gdf = gpd.GeoDataFrame(
    litter_df, geometry=gpd.points_from_xy(litter_df.long, litter_df.lat), crs="EPSG:4326"
)

# Save processed litter data (optional)
litter_gdf.to_csv("southampton_litter_processed.csv", index=False)
print("Step 2 completed: Litter data processed and saved to litter_processed.csv")


Step 2 completed: Litter data processed and saved to litter_processed.csv


In [32]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from scipy.spatial import cKDTree

# Load OA and litter data
OA_df = pd.read_csv("UK-OA-Population.csv")
litter_df = pd.read_csv("southampton_litter_processed.csv")

# Convert both to GeoDataFrames
OA_gdf = gpd.GeoDataFrame(
    OA_df, geometry=gpd.points_from_xy(OA_df.Long, OA_df.Lat), crs="EPSG:4326"
)

# litter_gdf = gpd.GeoDataFrame(
#     litter_df, geometry=gpd.points_from_xy(litter_df.long, litter_df.lat), crs="EPSG:4326"
# )
litter_gdf = gpd.GeoDataFrame(
    litter_df, geometry=gpd.points_from_xy(litter_df.category_center_long, litter_df.category_center_lat), crs="EPSG:4326"
)

# Step 3: Find the Nearest OA for Each Litter Location
def find_nearest_OA(litter_gdf, OA_gdf):
    # Convert GeoDataFrames to numpy arrays for efficient spatial indexing
    litter_coords = list(zip(litter_gdf.geometry.x, litter_gdf.geometry.y))
    OA_coords = list(zip(OA_gdf.geometry.x, OA_gdf.geometry.y))

    # Use KDTree for efficient nearest neighbor search
    tree = cKDTree(OA_coords)
    distances, indices = tree.query(litter_coords, k=1)  # Find the nearest OA

    # Add nearest OA to the litter DataFrame
    litter_gdf["OA2021"] = OA_gdf.iloc[indices]["OA2021"].values
    litter_gdf["OA_lat"] = OA_gdf.iloc[indices]["Lat"].values
    litter_gdf["OA_lon"] = OA_gdf.iloc[indices]["Long"].values
    litter_gdf["Population"] = OA_gdf.iloc[indices]["Population"].values
    litter_gdf["distance_m"] = distances * 111320  # Convert degrees to meters

    return litter_gdf

# Match litter locations to the nearest OA
result_gdf = find_nearest_OA(litter_gdf, OA_gdf)

# Save the results to a CSV file
result_gdf.drop(columns="geometry").to_csv("southampton_litter_with_OA&Population.csv", index=False)
print("Step 3 completed: Results saved to southampton_litter_with_OA&Population.csvv")


Step 3 completed: Results saved to southampton_litter_with_OA&Population.csvv


In [ ]:
#################  Litter per Population #################

import pandas as pd

def add_litter_count_column(file_path, output_file):
    # Load the dataset
    df = pd.read_csv(file_path)
    
    # Ensure Population column is numeric
    df['Population'] = pd.to_numeric(df['Population'], errors='coerce')
    
    # Count occurrences of each material category
    litter_counts = df.groupby('material_category_by_location')['litterId'].count()
    
    # Map counts back to the original dataset
    df['litter_count_by_category'] = df['material_category_by_location'].map(litter_counts)
    
    # Count occurrences of litter_count_by_category per OA2021, considering unique material_category_by_location
    unique_counts = df.drop_duplicates(subset=['OA2021', 'material_category_by_location'])
    oa_litter_counts = unique_counts.groupby('OA2021')['litter_count_by_category'].sum()
    
    # Map counts back to the original dataset
    df['total_litter_by_OA2021'] = df['OA2021'].map(oa_litter_counts)
    
    # Compute litter per population ratio and ensure only one row per OA2021 gets a value
    df['litter_per_population'] = 0  # Default value
    population_map = df.drop_duplicates(subset=['OA2021'])[['OA2021', 'Population']].set_index('OA2021')['Population']
    unique_counts['litter_per_population'] = unique_counts['OA2021'].map(oa_litter_counts).astype(float) / unique_counts['OA2021'].map(population_map).astype(float)
    
    # Map back to the original dataframe
    df.loc[df['OA2021'].isin(unique_counts['OA2021']), 'litter_per_population'] = unique_counts['litter_per_population']
    
    # Save the updated dataset
    df.to_csv(output_file, index=False)
    print(f"Updated dataset saved to {output_file}")

# Example usage
file_path = 'southampton_litter_with_OA&Population.csv'  # Replace with actual filename
output_file = 'southampton_litter_with_OA-per-Population.csv'
add_litter_count_column(file_path, output_file)


Updated dataset saved to southampton_litter_with_OA-per-Population.csv


C:\Users\Rahim\AppData\Local\Temp\ipykernel_11292\2417103227.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_counts['litter_per_population'] = unique_counts['OA2021'].map(oa_litter_counts).astype(float) / unique_counts['OA2021'].map(population_map).astype(float)
C:\Users\Rahim\AppData\Local\Temp\ipykernel_11292\2417103227.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.00447427 0.00518135 0.00310559 ... 0.02205882        nan 0.02205882]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['OA2021'].isin(unique_counts['OA2021']), 'litter_per_population'] = unique_counts['litter_per_population']


In [1]:
###########   Litter per Population by considering year ##################

import pandas as pd

def update_litter_analysis(file_path, output_file):
    # Load the dataset
    df = pd.read_csv(file_path)
    
    # Ensure Population column is numeric
    df['Population'] = pd.to_numeric(df['Population'], errors='coerce')
    
    # Preserve original timestamp for debugging
    df['original_timestamp'] = df['timestamp']
    
    # Try parsing timestamps with different formats
    def parse_timestamp(ts):
        try:
            return pd.to_datetime(ts, format='%m/%d/%Y %H:%M:%S')  # Try full format
        except ValueError:
            try:
                return pd.to_datetime(ts, format='%m/%d/%Y %H:%M')  # Try without seconds
            except ValueError:
                return pd.NaT  # If both fail, return NaT
    
    df['timestamp'] = df['original_timestamp'].apply(parse_timestamp)
    
    # Identify problematic timestamps before replacement
    missing_timestamps = df[df['timestamp'].isna()]['original_timestamp'].unique()
    print("Problematic timestamps that could not be converted:")
    print(missing_timestamps[:10])  # Print only the first 10 for inspection
    
    # Fill missing timestamps with 'Unknown' for debugging
    df['timestamp'].fillna("Unknown", inplace=True)
    
    # Extract year where timestamp is valid
    df['year'] = pd.to_datetime(df['timestamp'], errors='coerce').dt.year
    
    # Count occurrences of each material category
    litter_counts = df.groupby('material_category_by_location')['litterId'].count()
    
    # Map counts back to the original dataset
    df['litter_count_by_category'] = df['material_category_by_location'].map(litter_counts)
    
    # Count occurrences of litter_count_by_category per OA2021, considering unique material_category_by_location
    unique_counts = df.drop_duplicates(subset=['OA2021', 'material_category_by_location'])
    oa_litter_counts = unique_counts.groupby('OA2021')['litter_count_by_category'].sum()
    
    # Multiply total_litter_by_OA2021 by 1.7
    df['total_litter_by_OA2021'] = df['OA2021'].map(oa_litter_counts) * 2.3
    
    # Count unique years per OA2021
    year_counts = df.groupby('OA2021')['year'].nunique()
    df['year_count'] = df['OA2021'].map(year_counts)
    
    # Compute Population * year_count and store in a new column
    population_map = df.drop_duplicates(subset=['OA2021'])[['OA2021', 'Population']].set_index('OA2021')['Population']
    df['population_times_year'] = df['OA2021'].map(population_map) * df['year_count']
    
    # Compute litter per population using Population * year_count
    df['litter_per_population'] = 0  # Default value
    
    unique_counts['litter_per_population'] = unique_counts['OA2021'].map(oa_litter_counts).astype(float) * 1.7 / df['population_times_year']
    
    # Map back to the original dataframe
    df.loc[df['OA2021'].isin(unique_counts['OA2021']), 'litter_per_population'] = unique_counts['litter_per_population']
    
    # Save the updated dataset
    df.to_csv(output_file, index=False)
    print(f"Updated dataset saved to {output_file}")
    
    # Debugging: Print missing timestamps count
    missing_timestamps_count = df[df['timestamp'] == "Unknown"].shape[0]
    print(f"Missing timestamps count: {missing_timestamps_count}")
    if missing_timestamps_count > 0:
        print("Example of missing timestamps:")
        print(df[df['timestamp'] == "Unknown"].head())

# Example usage
file_path = 'southampton_litter_with_OA&Population.csv'  # Replace with actual filename
output_file = 'southampton_litter_with_OA-per-Population-based-year.csv'
update_litter_analysis(file_path, output_file)


Problematic timestamps that could not be converted:
[]
Updated dataset saved to southampton_litter_with_OA-per-Population-based-year.csv
Missing timestamps count: 0


/var/folders/g3/07d8ggrx03ld_fj9md65wkth0000gn/T/ipykernel_89655/3289506877.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_counts['litter_per_population'] = unique_counts['OA2021'].map(oa_litter_counts).astype(float) * 1.7 / df['population_times_year']
/var/folders/g3/07d8ggrx03ld_fj9md65wkth0000gn/T/ipykernel_89655/3289506877.py:65: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.00760626 0.00880829 0.0052795  ... 0.0375            nan 0.0375    ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['OA2021'].isin(unique_counts['OA2021']), 'litter_per_population'] = unique_counts['litter_per_population']
